# Mo_ma_c

In [1]:
import sys
sys.path.append('/home/phzd/AI/bidd-molmap_v1_3')
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import auc as calculate_auc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.utils import shuffle 
from joblib import load, dump
import numpy as np
import pandas as pd
import os,time,json
from molmap import feature,loadmap
from scipy.stats.stats import pearsonr
from argparse import ArgumentParser
import xgboost
import shap

In [2]:
def PRC_AUC(y_true, y_score):
	precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
	auc = calculate_auc(recall, precision)
	return auc

def ROC_AUC(y_true, y_score):
	auc = roc_auc_score(y_true, y_score)
	return auc

In [3]:
df=pd.read_csv('ChblClass7758_fillNullWith0.csv')
Y0 = df['active'].astype('int').values
len(Y0)

7758

In [4]:
X0=pd.read_csv('Mo_ma_c.csv')
len(X0)

7758

In [5]:
def random_split(df, random_state, split_size = [0.8, 0.1, 0.1]):
	base_indices = np.arange(len(df)) 
	base_indices = shuffle(base_indices, random_state = random_state) 
	nb_test = int(len(base_indices) * split_size[2]) 
	nb_val = int(len(base_indices) * split_size[1]) 
	test_idx = base_indices[0:nb_test] 
	valid_idx = base_indices[(nb_test):(nb_test+nb_val)] 
	train_idx = base_indices[(nb_test+nb_val):len(base_indices)] 
	print(len(train_idx), len(valid_idx), len(test_idx)) 
	return train_idx, valid_idx, test_idx 

In [6]:
seed = 1
X0=X0.values
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx] 

6208 775 775
6208 775 775


In [6]:
import json
with open('params_class','r') as f:
    best_param=json.load(f)

In [7]:
best_param['params']

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.66,
 'gamma': 0,
 'gpu_id': 0,
 'learning_rate': 0.05,
 'max_delta_step': 1,
 'max_depth': 8,
 'min_child_weight': 5,
 'missing': None,
 'n_estimators': 1500,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 123,
 'silent': None,
 'subsample': 0.53,
 'tree_method': 'gpu_hist',
 'verbosity': 1}

In [8]:
use_param=best_param['params']
for item in ['gpu_id','tree_method','missing']:
    use_param.pop(item)
use_param.update({'n_jobs': -1})

In [9]:
import time
clf=xgboost.XGBClassifier(**use_param)

In [11]:
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[10:19:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 124.2902364730835


In [12]:
clf.predict_proba(X_test)

array([[5.3459406e-04, 9.9946541e-01],
       [1.4466226e-02, 9.8553377e-01],
       [8.1857109e-01, 1.8142889e-01],
       ...,
       [2.4056196e-02, 9.7594380e-01],
       [6.6141033e-01, 3.3858967e-01],
       [3.1253576e-01, 6.8746424e-01]], dtype=float32)

In [13]:
clf.predict_proba(X_valid)

array([[0.9627813 , 0.03721868],
       [0.21463841, 0.7853616 ],
       [0.99759376, 0.00240623],
       ...,
       [0.6369529 , 0.36304715],
       [0.948519  , 0.05148104],
       [0.00179553, 0.99820447]], dtype=float32)

In [14]:
clf.predict_proba(X_test)[:3,1]

array([0.9994654, 0.9855338, 0.1814289], dtype=float32)

In [15]:
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

valid_roc_auc, test_roc_auc =  0.9264889336016097 0.925734726230641


In [45]:
seed = 2
X0=X0.values
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx] 

6208 775 775
6208 775 775


In [49]:
clf=''
clf=xgboost.XGBClassifier(**use_param)

In [50]:
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[15:41:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 136.83449602127075
valid_roc_auc, test_roc_auc =  0.9182776683680989 0.9221967963386726


In [10]:
seeds = [4,8,16,32,64,128,256,512]
X0=X0.values
for seed in seeds:
    train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
    train_idx = [i for i in train_idx if i < len(df)]
    valid_idx = [i for i in valid_idx if i < len(df)]	
    test_idx = [i for i in test_idx if i < len(df)]	 
    X = X0[train_idx]; y = Y0[train_idx]
    X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
    X_test = X0[test_idx]; y_test = Y0[test_idx]
    clf=''
    clf=xgboost.XGBClassifier(**use_param)
    time1=time.time()
    model=clf.fit(X, y)
    time2=time.time()
    time_fit=time2-time1
    print(f"fit time is: {time_fit}")
    test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
    valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
    print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[15:46:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 121.98517608642578
valid_roc_auc, test_roc_auc =  0.9240026773761714 0.9277455278612217
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[15:48:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 131.54975700378418
valid_roc_auc, test_roc_auc =  0.9271458967383986 0.9350349584658534
6208 775 775


...

[15:57:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 134.40759181976318
valid_roc_auc, test_roc_auc =  0.9198531110295816 0.9177789046653144
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[15:59:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 139.34256625175476
valid_roc_auc, test_roc_auc =  0.9172702883970489 0.9368251268761472
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:01:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 135.3521158695221
valid_roc_auc, test_roc_auc =  0.9272973955668076 0.9255362614913177


# Mo_Pu

In [16]:
X0=pd.read_csv('Mo_Pu_c.csv')
len(X0)

7758

In [17]:
def random_split(df, random_state, split_size = [0.8, 0.1, 0.1]):
	base_indices = np.arange(len(df)) 
	base_indices = shuffle(base_indices, random_state = random_state) 
	nb_test = int(len(base_indices) * split_size[2]) 
	nb_val = int(len(base_indices) * split_size[1]) 
	test_idx = base_indices[0:nb_test] 
	valid_idx = base_indices[(nb_test):(nb_test+nb_val)] 
	train_idx = base_indices[(nb_test+nb_val):len(base_indices)] 
	print(len(train_idx), len(valid_idx), len(test_idx)) 
	return train_idx, valid_idx, test_idx 

In [18]:
seed = 1
X0=X0.values
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx]

6208 775 775
6208 775 775


In [19]:
import time
clf=xgboost.XGBClassifier(**use_param)
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[10:48:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 168.5464906692505


In [23]:
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

valid_roc_auc, test_roc_auc =  0.9305197853789403 0.9266696475753018


In [11]:
X0=pd.read_csv('Mo_Pu_c.csv')
len(X0)

7758

In [12]:
seeds = [1,2,4,8,16,32,64,128,256,512]
X0=X0.values
for seed in seeds:
    train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
    train_idx = [i for i in train_idx if i < len(df)]
    valid_idx = [i for i in valid_idx if i < len(df)]	
    test_idx = [i for i in test_idx if i < len(df)]	 
    X = X0[train_idx]; y = Y0[train_idx]
    X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
    X_test = X0[test_idx]; y_test = Y0[test_idx]
    clf=''
    clf=xgboost.XGBClassifier(**use_param)
    time1=time.time()
    model=clf.fit(X, y)
    time2=time.time()
    time_fit=time2-time1
    print(f"fit time is: {time_fit}")
    test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
    valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
    print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:05:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 180.3265233039856
valid_roc_auc, test_roc_auc =  0.9305197853789403 0.9266696475753018
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:08:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 184.42662978172302
valid_roc_auc, test_roc_auc =  0.9196508714815856 0.922142634693242
6208 775 775


...

[16:26:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 194.91742539405823
valid_roc_auc, test_roc_auc =  0.9212900683488919 0.9215382014874915
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:30:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 199.99020648002625
valid_roc_auc, test_roc_auc =  0.9142655935613682 0.9401455026455027
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:33:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 198.0845398902893
valid_roc_auc, test_roc_auc =  0.9305832368229686 0.9246033367381682


# Mo_Pu_Ma

In [24]:
X0=pd.read_csv('Mo_Pu_ma_c.csv')
len(X0)

7758

In [25]:
def random_split(df, random_state, split_size = [0.8, 0.1, 0.1]):
	base_indices = np.arange(len(df)) 
	base_indices = shuffle(base_indices, random_state = random_state) 
	nb_test = int(len(base_indices) * split_size[2]) 
	nb_val = int(len(base_indices) * split_size[1]) 
	test_idx = base_indices[0:nb_test] 
	valid_idx = base_indices[(nb_test):(nb_test+nb_val)] 
	train_idx = base_indices[(nb_test+nb_val):len(base_indices)] 
	print(len(train_idx), len(valid_idx), len(test_idx)) 
	return train_idx, valid_idx, test_idx 

In [26]:
seed = 1
X0=X0.values
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx]

6208 775 775
6208 775 775


In [27]:
import time
clf=xgboost.XGBClassifier(**use_param)
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[10:58:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 179.50817227363586


In [31]:
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

valid_roc_auc, test_roc_auc =  0.9319215291750502 0.926208961695324


In [13]:
X0=pd.read_csv('Mo_Pu_ma_c.csv')
len(X0)

7758

In [14]:
seeds = [1,2,4,8,16,32,64,128,256,512]
X0=X0.values
for seed in seeds:
    train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
    train_idx = [i for i in train_idx if i < len(df)]
    valid_idx = [i for i in valid_idx if i < len(df)]	
    test_idx = [i for i in test_idx if i < len(df)]	 
    X = X0[train_idx]; y = Y0[train_idx]
    X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
    X_test = X0[test_idx]; y_test = Y0[test_idx]
    clf=''
    clf=xgboost.XGBClassifier(**use_param)
    time1=time.time()
    model=clf.fit(X, y)
    time2=time.time()
    time_fit=time2-time1
    print(f"fit time is: {time_fit}")
    test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
    valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
    print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:39:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 195.61753797531128
valid_roc_auc, test_roc_auc =  0.9319215291750502 0.926208961695324
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:42:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 210.6504065990448
valid_roc_auc, test_roc_auc =  0.9192355612716531 0.9248371765534237
6208 775 775


...

[17:03:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 211.05363392829895
valid_roc_auc, test_roc_auc =  0.922037422037422 0.9201791751183231
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:07:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 217.2484211921692
valid_roc_auc, test_roc_auc =  0.9182897384305836 0.9428989849908218
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:10:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 216.80072569847107
valid_roc_auc, test_roc_auc =  0.9320780252632712 0.9220292815798433


# Pu_Ma

In [32]:
X0=pd.read_csv('Pu_ma_c.csv')
len(X0)

7758

In [33]:
def random_split(df, random_state, split_size = [0.8, 0.1, 0.1]):
	base_indices = np.arange(len(df)) 
	base_indices = shuffle(base_indices, random_state = random_state) 
	nb_test = int(len(base_indices) * split_size[2]) 
	nb_val = int(len(base_indices) * split_size[1]) 
	test_idx = base_indices[0:nb_test] 
	valid_idx = base_indices[(nb_test):(nb_test+nb_val)] 
	train_idx = base_indices[(nb_test+nb_val):len(base_indices)] 
	print(len(train_idx), len(valid_idx), len(test_idx)) 
	return train_idx, valid_idx, test_idx 

In [34]:
seed = 1
X0=X0.values
train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
train_idx = [i for i in train_idx if i < len(df)]
valid_idx = [i for i in valid_idx if i < len(df)]	
test_idx = [i for i in test_idx if i < len(df)]	
print(len(train_idx), len(valid_idx), len(test_idx)) 
X = X0[train_idx]; y = Y0[train_idx]
X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
X_test = X0[test_idx]; y_test = Y0[test_idx]

6208 775 775
6208 775 775


In [35]:
import time
clf=xgboost.XGBClassifier(**use_param)
time1=time.time()
model=clf.fit(X, y)
time2=time.time()
time_fit=time2-time1
print(f"fit time is: {time_fit}")

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[11:22:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 58.55386304855347


In [39]:
test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

valid_roc_auc, test_roc_auc =  0.9317706237424547 0.9190242944053765


In [15]:
X0=pd.read_csv('Pu_ma_c.csv')
len(X0)

7758

In [16]:
seeds = [1,2,4,8,16,32,64,128,256,512]
X0=X0.values
for seed in seeds:
    train_idx, valid_idx, test_idx = random_split(df,random_state=seed)
    train_idx = [i for i in train_idx if i < len(df)]
    valid_idx = [i for i in valid_idx if i < len(df)]	
    test_idx = [i for i in test_idx if i < len(df)]	 
    X = X0[train_idx]; y = Y0[train_idx]
    X_valid = X0[valid_idx];y_valid = Y0[valid_idx]
    X_test = X0[test_idx]; y_test = Y0[test_idx]
    clf=''
    clf=xgboost.XGBClassifier(**use_param)
    time1=time.time()
    model=clf.fit(X, y)
    time2=time.time()
    time_fit=time2-time1
    print(f"fit time is: {time_fit}")
    test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
    valid_roc_auc = ROC_AUC(y_valid,clf.predict_proba(X_valid)[:, 1])
    print('valid_roc_auc, test_roc_auc = ',valid_roc_auc, test_roc_auc)

6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:15:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 63.1480393409729
valid_roc_auc, test_roc_auc =  0.9317706237424547 0.9190242944053765
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:16:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 70.31933617591858
valid_roc_auc, test_roc_auc =  0.9158126013155956 0.9175118140089097
6208 775 775


...

[17:24:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 70.93473148345947
valid_roc_auc, test_roc_auc =  0.9115235144646908 0.9137423935091278
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:25:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 69.84852480888367
valid_roc_auc, test_roc_auc =  0.9075586854460094 0.9331740632761041
6208 775 775


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:26:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time is: 70.8059868812561
valid_roc_auc, test_roc_auc =  0.9195137224272132 0.9086925434116445
